<div style='color: #690027;' markdown="1">
    <h1>Overfitting</h1> 
</div>

<div style='color: #690027;' markdown="1">
    <h2>Overfitting tegengaan</h2> 
</div>

Wanneer een netwerk te gefixeerd geraakt op de training data zal het minder goed presteren op ongeziene data, het netwerk leert dan als het ware de training data van buiten. Dit wordt <b>overfitting</b> genoemd. Wanneer het netwerk nog kan bijleren  spreekt men van <b>underfitting</b>. Volgende figuur stelt beide concepten visueel voor, een diep neuraal netwerk zoekt als het ware een functie die de invoer afbeeld op de juiste uitvoer en die goed genoeg generaliseert.

<img src="../.images/IntroductieDeepLearning/overfitting.jpg"/>

Overfitting is één van de meest voorkomende problemen bij het bouwen van een diep neuraal netwerk, gelukkig bestaan er verschillende technieken om overfitting tegen te gaan.

<b><u>Meer training data</u></b>

Hoe meer training data je hebt hoe minder waarschijnlijk het is dat je model snel zal overfitten. De training data zal bijdragen aan de aanpassing van de gewichten van het netwerk tijdens het leren. Wanneer er meer training data is zal het model dus minder snel al deze data vanbuiten kunnen leren.

<b><u>Simpeler netwerk</u></b>

Een diep netwerk met veel lagen en gewichten zal veel informatie over de training data kunnen opslaan en leidt tot overfitting. Een te simpel netwerk zal dan weer niet genoeg leren uit de data. Het is vaak aangewezen om met een relatief simpel netwerk te starten en geleidelijk extra lagen of dimensies toe te voegen.

<b><u>Data augmentatie</u></b>

Hoe meer training data hoe beter, soms heb je echter niet heel veel training data. Met data augmentatie ga je je training data aanpassen zodat het netwerk steeds andere data te zien krijgt. Bij afbeeldingen kan je bijvoorbeeld de afbeelding een bepaald aantal graden draaien of spiegelen over de x of de y-as. Je kan de intensiteit aanpassen, de afbeelding naar links, rechts, boven of onder verschuiven, enzovoort. Volgende afbeelding toont data augmentatie op een afbeelding van een stoma. Er is enkel gespiegeld over de x en y-as en geroteerd. Verschuiven zou hier geen goed idee zijn want het model herkent afbeeldingen met een stoma in het middelpunt.

<img src="../.images/IntroductieDeepLearning/data_augmentatie.jpg" width="300"/>

<b><u>Dropout</u></b>

Een zeer efficiënte en veelgebruikte methode van regularisatie is het toevoegen van dropout. Door dropout toe te voegen zal een willekeurig gedeelte van de uitvoer van een laag herleidt worden tot 0. Door steeds andere uitvoer te kiezen zal het netwerk geen patronen leren die niet belangrijk zijn. Het percentage van de uitvoer dat naar 0 herleid moet worden wordt de <b>dropout rate</b> genoemd, bij de volgende voorbeelden hebben we steeds een dropout rate van 0.5 gekozen. Volgende afbeelding toont hoe dropout werk in een diep neuraal netwerk.

<img src="../.images/IntroductieDeepLearning/dropout.jpg"/>

Als voorbeeld gaan we opnieuw een netwerk samenstellen, deze keer wordt er echter gebruik gemaakt van data augmentatie van de training data en wordt er na elke feedforward laag dropout toegevoegd.

In [ ]:
import imp
with open('../.scripts/diep_neuraal_netwerk.py', 'rb') as fp:
    diep_neuraal_netwerk = imp.load_module('.scripts', fp, '../.scripts/diep_neuraal_netwerk.py', ('.py', 'rb', imp.PY_SOURCE))
diep_neuraal_netwerk.kies_netwerk()

Na het kiezen van de netwerk architectuur en enkele netwerk parameters kunnen we opnieuw het netwerk visualiseren met volgende codecel.

In [ ]:
diep_neuraal_netwerk.toon_netwerk(regularization=True)

<div style='color: #690027;' markdown="1">
    <h2>Resultaten</h2> 
</div>

"Leg uit wat er anders moet zijn met regularisatie"

In [ ]:
print("Gekozen netwerk met regularisatie\n")
diep_neuraal_netwerk.toon_accuracy(regularization=True)
diep_neuraal_netwerk.toon_accuracy_grafiek(regularization=True)

print("Gekozen netwerk zonder regularisatie\n")
diep_neuraal_netwerk.toon_accuracy(regularization=False)
diep_neuraal_netwerk.toon_accuracy_grafiek(regularization=False)

We kunnen opnieuw de voorspellingen van het netwerk met regularisatie weergeven voor enkele voorbeelden met volgende codecel.

In [ ]:
diep_neuraal_netwerk.toon_afbeeldingen(regularization=True)

<div style='color: #690027;' markdown="1">
    <h2>Stomata detectie in een volledige afbeelding</h2> 
</div>

"Detectie in afbeelding uitleggen + clustering"

<img src="../.images/IntroductieDeepLearning/sliding_window.gif"/>

In [ ]:
diep_neuraal_netwerk.vind_stomata(regularization=True)